# Top

Before `git add`:

* Edit, Clear Outputs of All Cells

In [ ]:
import fullcontrol as fc

# Utility Functions

In [ ]:
def spiral(outer_size, num_turns, cw):
    """
    Generate a square spiral that reaches `outer_size` after `num_turns`.
    Returns a list of (x, y) coordinates.
    """
    if num_turns < 1:
        return [(0, 0)]

    num_segments = 4 * num_turns + 1

    step = outer_size / num_turns / 2
    x, y = 0, 0
    dx, dy = 1, 0  # initial direction: right
    coords = [(x, y)]

    for i in range(num_segments):
        f = (i // 2 + 1 * (i < num_segments - 1)) * step
        x += dx * f
        y += dy * f
        coords.append((x, y))
        if cw:
            dx, dy = dy, -dx  # rotate 90 degrees CW
        else:
            dx, dy = -dy, dx  # rotate 90 degrees CCW

    return coords

In [ ]:
# Inspiration from:
# spiralXY(centre: Point, start_radius: float, end_radius: float, start_angle: float, n_turns: float, segments: int, cw: bool=False) -> list
def squareSpiral(center, size: float, n_turns: int, cw=False):
    steps = []
    for p in spiral(size, n_turns, cw):
        steps.append(fc.Point(x=p[0] + center.x, y=p[1] + center.y, z=center.z))
    return steps

# Features

## Extruder on/off

```
# Travel
steps.extends(fc.travel_to(fc.Point(...)))

# Manuel
steps.append(fc.Extruder(on=False))
```

## Print Speed

```
steps.append(fc.Printer(print_speed=750))
```

## Fan

```
steps.append(fc.Fan(speed_percent=50))
```

# Design

## Parameters

In [ ]:
# Parameters

design_name = 'my_design'
nozzle_temp = 220
bed_temp = 60
print_speed = 1000
fan_percent = 100
printer_name='ender_3' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0

EW = 0.4 # extrusion width
EH = 0.2 # extrusion height (and layer height)

## Simple

In [ ]:
steps = []

steps.extend(fc.travel_to(fc.Point(x=10, y=10, z=.2)))
steps.append(fc.Point(x=110))
steps.append(fc.Point(y=110))
steps.append(fc.Point(x=10, y=10))

steps

## Complex

In [ ]:
steps = []

steps.append(fc.Fan(speed_percent=0))  # Fan OFF for first layer

z = EH

steps.append(fc.Point(x=2, y=2, z=z))
steps.append(fc.Extruder(on=True))
steps.append(fc.Point(y=200))
steps.append(fc.Extruder(on=False))

pt = fc.Point(x=20, y=20)
steps.append(pt)

steps.append(fc.Extruder(on=True))
steps.extend(fc.rectangleXY(pt, 100, 10, cw=True))
steps.append(fc.Extruder(on=False))

pt = fc.Point(x=40, y=100)
steps.append(pt)
steps.append(fc.Extruder(on=True))
steps.extend(fc.spiralXY(pt, 1, 10, 0, 10, 1000, True))
steps.append(fc.Extruder(on=False))

pt = fc.Point(x=20, y=40)
steps.append(pt)
steps.append(fc.Extruder(on=True))
steps.extend(fc.squarewaveXY(pt, fc.Vector(x=1, y=0, z=0), 10, 2, 30))
steps.append(fc.Extruder(on=False))

pt = fc.Point(x=100, y=100)
steps.append(pt)
steps.append(fc.Extruder(on=True))
steps.extend(squareSpiral(pt, 80, 20, True))
# steps

# Visualization

In [ ]:
fc.transform(
    steps,
    'plot',
    fc.PlotControls(
        #neat_for_publishing=True,
        style='line',
        #style='tube',
        zoom=.9
    )
)

# GCODE

## Basic

In [ ]:
gcode = fc.transform(
    steps,
    'gcode',
    controls=fc.GcodeControls(
        printer_name='generic',  # or ender_3
        initialization_data={
            'extrusion_width': EW,
            'extrusion_height': EH
        }
    )
)
#print(gcode)

## Ender 3

In [ ]:
gcode = fc.transform(
    steps,
    'gcode',
    fc.GcodeControls(
        printer_name=printer_name,
        initialization_data={
            'primer': 'front_lines_then_y',
            'print_speed': print_speed,
            'nozzle_temp': nozzle_temp,
            'bed_temp': bed_temp,
            'fan_percent': fan_percent,
            'extrusion_width': EW,
            'extrusion_height': EH},
        save_as=design_name,
        include_date=False
    )
)

In [ ]:
for l in gcode.splitlines()[:80]:
    print(l)